#### Import Libraries

In [ ]:
import cv2                 
import numpy as np         
import os                  
from random import shuffle
import matplotlib.pyplot as plt
import glob as gb


In [ ]:
import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold


In [ ]:
from keras import backend as K

In [ ]:
from tensorflow.keras.applications import InceptionV3
from builtins import range, input
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, AveragePooling2D, Dropout
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.utils import to_categorical

#### Setting the dataset path


In [ ]:
#read DataSet
Image_Directory = "./Dataset/"

In [ ]:
#to get all image names in train file
Healthy_images = os.listdir(Image_Directory + "/Healthy_images")
Covid_images = os.listdir(Image_Directory + "/Covid_images")

In [ ]:
Labels={'Covid_images':0,'Healthy_images':1 }

#convert label to code
def getCode(label):
    return Labels[label]

#convert code to label 
def getLabel(n):
    for x,c in Labels.items():
        if n==c:
            return x
               
#Test        
print(getCode('Covid_images'))
print(getLabel(1))

#### Load Data

In [ ]:
sizeImage=224 # to resize the all image as same size

#to read all images from directory
def getData(Dir,sizeImage):
    X=[]
    y=[]
    for folder in  os.listdir(Dir) : #to get the file name 
        files = gb.glob(pathname= str( Dir  +"/" +folder+ '//*.png' )) #to get the images
        for file in files:
                picture=cv2.imread(file)
                imageArray=cv2.resize(picture,(sizeImage,sizeImage))
                X.append(list(imageArray))
                y.append(getCode(folder))
    X=np.array(X)
    y=np.array(y)
    return X,y

In [ ]:
#get train data
X, y = getData(Image_Directory,sizeImage)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
print("X_train Shape        ",X_train.shape)

In [ ]:
#Convert y_train to categorical
y_train=to_categorical(y_train,2)
print("y_train ",y_train.shape)

#Convert y_train to categorical
y_test=to_categorical(y_test,2)
print("y_test ",y_test.shape)

In [ ]:
train_datagen = ImageDataGenerator(rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=True)

#### Build Model

In [ ]:
pre_trained_model = InceptionV3(weights="imagenet", include_top=False,
    input_tensor=Input(shape=(224, 224, 3)))

outputs = pre_trained_model.output
outputs = Flatten(name="flatten")(outputs)
outputs = Dropout(0.5)(outputs)
outputs = Dense(2, activation="softmax")(outputs)

model = Model(inputs=pre_trained_model.input, outputs=outputs)

for layer in pre_trained_model.layers:
    layer.trainable = False

model.compile(
        loss='categorical_crossentropy', 
        optimizer='adam', 
        metrics=['accuracy', 
                 tf.keras.metrics.AUC(name='auc'), 
                 tf.keras.metrics.Precision(name='precision'), 
                        tf.keras.metrics.Recall(name='recall')]
)

#### K-fold Cross Validation

In [ ]:
#K-fold Cross Validation model evaluation
num_folds = 5
fold_no = 1
epochs = 20
acc_per_fold = []
loss_per_fold = []

kfold = KFold(n_splits=num_folds, shuffle=True, random_state=None)

In [ ]:
lr_reduce = ReduceLROnPlateau(monitor='accuracy', factor=0.1, min_delta=0.0001, patience=1, verbose=1)

filepath="inc.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='accuracy', verbose=1, save_best_only=True, mode='max')

In [ ]:
score_list = []

for train, test in kfold.split(X_train, y_train):
    
    train_generator =train_datagen.flow(
         X_train[train], y_train[train],
         batch_size= 256
    )

    test_generator =train_datagen.flow(
         X_train[test], y_train[test],
         batch_size= 50
    )  

    history = model.fit(train_generator,steps_per_epoch=50,callbacks=[lr_reduce,checkpoint],
                        validation_data=(X_test, y_test),
                    validation_steps=len(X_test) / 32,
             epochs=epochs)

    scores = model.evaluate(test_generator)
    score_list.append(scores)

    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%; {model.metrics_names[2]} of {scores[2]}; {model.metrics_names[3]} of {scores[3]}; {model.metrics_names[4]} of {scores[4]}')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
    
    fold_no = fold_no + 1

In [ ]:
for s in score_list:
    print(s[1]*100)

#### Model Evaluation

In [ ]:
test_generator =train_datagen.flow(
     X_test, y_test,
     batch_size= 50,
)

In [ ]:
#Evaluate Model
model.evaluate(test_generator)

In [ ]:
#prediction
pred = model.predict(test_generator)

In [ ]:
batch_size = 32
y_pred = model.predict(X_test, batch_size=batch_size)
y_pred_bin = np.argmax(y_pred, axis=1)
y_test_bin = np.argmax(y_test, axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix, roc_curve
fpr, tpr, thresholds = roc_curve(y_test_bin, y_pred_bin)
plt.plot(fpr, tpr,linewidth=2)
plt.plot([0,1],[0,1],'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.rcParams['font.size'] = 12
plt.title('ROC curve for our model')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.grid(True)
plt.savefig('inception_roc.png')

In [ ]:
import seaborn as sns
def plot_confusion_matrix(normalize):
    classes = ['Covid_images','Healthy_images']
    tick_marks = [0.5,1.5]
    cn = confusion_matrix(y_test_bin, y_pred_bin,normalize=normalize)
    sns.heatmap(cn,cmap=plt.cm.Blues,annot=True)
    plt.xticks(tick_marks, classes)
    plt.yticks(tick_marks, classes)
    plt.title('Confusion Matrix')
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

print('Confusion Matrix without Normalization')
plot_confusion_matrix(normalize=None)

print('Confusion Matrix with Normalized Values')
plot_confusion_matrix(normalize='true')

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test_bin, y_pred_bin))